# Alternative PygmalionAI notebook for Google Colab

This notebook is an *unofficial* method to run the [PygmalionAI LLMs](https://huggingface.co/PygmalionAI) (NSFW) using **oobabooga**'s [text-generation-webui](https://github.com/oobabooga/text-generation-webui) project in chat mode.

Run all the cells and a public gradio URL will appear at the bottom in around 12 minutes.

* Optionally, you can activate automatic translation using Google Translate to talk to the bot in your own language.
* To create your own custom characters, you can use the [Pygmalion JSON character creator](https://oobabooga.github.io/character-creator.html)

Original author: https://github.com/81300

In [ ]:
#@title 1. Keep this tab alive to prevent Colab from disconnecting you { display-mode: "form" }

# From: https://github.com/henk717/KoboldAI

%%html
<b>Press play on the music player to keep the tab alive (Uses only 13MB of data)</b><br/>
<audio src="https://henk.tech/colabkobold/silence.m4a" controls>

In [ ]:
#@title 2. Install the requirements

%cd /tmp
!wget -c https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
%env PYTHONPATH=
!which conda 2>/dev/null || bash Miniconda3-latest-Linux-x86_64.sh -b -p /usr/local -f
%env CI=true
!conda init -q bash
!source /usr/local/bin/activate && conda update -y -q -c defaults --all
!source /usr/local/bin/activate && conda create -y -q -n textgen
!source /usr/local/bin/activate && conda activate textgen && conda install -y -q -c defaults conda python
!source /usr/local/bin/activate && conda activate textgen && conda install -y -q -c pytorch -c nvidia pytorch torchvision torchaudio pytorch-cuda=11.7
!source /usr/local/bin/activate && conda clean -y -q --all

# Download Github repo
%cd /content
!git clone https://github.com/oobabooga/text-generation-webui
!ln -s text-generation-webui/models .
!ln -s text-generation-webui/characters .
!ln -s text-generation-webui/presets .
%rm -r sample_data
%cd text-generation-webui
!wget https://oobabooga.github.io/settings-colab.json -O settings-colab-template.json

# Install requirements
!source /usr/local/bin/activate && conda activate textgen && pip install --no-cache-dir --progress-bar=off --root-user-action=ignore -r requirements.txt diffusers
!source /usr/local/bin/activate && conda activate textgen && pip install --no-cache-dir --progress-bar=off --root-user-action=ignore -r extensions/google_translate/requirements.txt

In [ ]:
#@title 3. Start the web UI

import json

model = "Pygmalion 6B-original (yields better results)" #@param ["Pygmalion 6B-original (yields better results)", "Pygmalion 6B", "Pygmalion 6B experimental", "Pygmalion 2.7B", "Pygmalion 1.3B", "Pygmalion 350M"] {allow-input: false}

models = {
    "Pygmalion 6B-original (yields better results)": ("PygmalionAI", "pygmalion-6b", "b8344bb4eb76a437797ad3b19420a13922aaabe1", "pygmalion-6b_b8344bb4eb76a437797ad3b19420a13922aaabe1"),
    "Pygmalion 6B": ("PygmalionAI", "pygmalion-6b", "main", "pygmalion-6b"),
    "Pygmalion 6B experimental": ("PygmalionAI", "pygmalion-6b", "dev", "pygmalion-6b_dev"),
    "Pygmalion 2.7B": ("PygmalionAI", "pygmalion-2.7b", "main", "pygmalion-2.7b"),
    "Pygmalion 1.3B": ("PygmalionAI", "pygmalion-1.3b", "main", "pygmalion-1.3b"),
    "Pygmalion 350M": ("PygmalionAI", "pygmalion-350m", "main", "pygmalion-350m")
}
huggingface_org, huggingface_repo, huggingface_branch, model_name = models[model]

# Download the model
![[ ! -f models/$model_name/config.json ]] && source /usr/local/bin/activate && conda activate textgen \
  && python download-model.py $huggingface_org/$huggingface_repo --branch $huggingface_branch
language_codes = {'Afrikaans': 'af', 'Albanian': 'sq', 'Amharic': 'am', 'Arabic': 'ar', 'Armenian': 'hy', 'Azerbaijani': 'az', 'Basque': 'eu', 'Belarusian': 'be', 'Bengali': 'bn', 'Bosnian': 'bs', 'Bulgarian': 'bg', 'Catalan': 'ca', 'Cebuano': 'ceb', 'Chinese (Simplified)': 'zh-CN', 'Chinese (Traditional)': 'zh-TW', 'Corsican': 'co', 'Croatian': 'hr', 'Czech': 'cs', 'Danish': 'da', 'Dutch': 'nl', 'English': 'en', 'Esperanto': 'eo', 'Estonian': 'et', 'Finnish': 'fi', 'French': 'fr', 'Frisian': 'fy', 'Galician': 'gl', 'Georgian': 'ka', 'German': 'de', 'Greek': 'el', 'Gujarati': 'gu', 'Haitian Creole': 'ht', 'Hausa': 'ha', 'Hawaiian': 'haw', 'Hebrew': 'iw', 'Hindi': 'hi', 'Hmong': 'hmn', 'Hungarian': 'hu', 'Icelandic': 'is', 'Igbo': 'ig', 'Indonesian': 'id', 'Irish': 'ga', 'Italian': 'it', 'Japanese': 'ja', 'Javanese': 'jw', 'Kannada': 'kn', 'Kazakh': 'kk', 'Khmer': 'km', 'Korean': 'ko', 'Kurdish': 'ku', 'Kyrgyz': 'ky', 'Lao': 'lo', 'Latin': 'la', 'Latvian': 'lv', 'Lithuanian': 'lt', 'Luxembourgish': 'lb', 'Macedonian': 'mk', 'Malagasy': 'mg', 'Malay': 'ms', 'Malayalam': 'ml', 'Maltese': 'mt', 'Maori': 'mi', 'Marathi': 'mr', 'Mongolian': 'mn', 'Myanmar (Burmese)': 'my', 'Nepali': 'ne', 'Norwegian': 'no', 'Nyanja (Chichewa)': 'ny', 'Pashto': 'ps', 'Persian': 'fa', 'Polish': 'pl', 'Portuguese (Portugal, Brazil)': 'pt', 'Punjabi': 'pa', 'Romanian': 'ro', 'Russian': 'ru', 'Samoan': 'sm', 'Scots Gaelic': 'gd', 'Serbian': 'sr', 'Sesotho': 'st', 'Shona': 'sn', 'Sindhi': 'sd', 'Sinhala (Sinhalese)': 'si', 'Slovak': 'sk', 'Slovenian': 'sl', 'Somali': 'so', 'Spanish': 'es', 'Sundanese': 'su', 'Swahili': 'sw', 'Swedish': 'sv', 'Tagalog (Filipino)': 'tl', 'Tajik': 'tg', 'Tamil': 'ta', 'Telugu': 'te', 'Thai': 'th', 'Turkish': 'tr', 'Ukrainian': 'uk', 'Urdu': 'ur', 'Uzbek': 'uz', 'Vietnamese': 'vi', 'Welsh': 'cy', 'Xhosa': 'xh', 'Yiddish': 'yi', 'Yoruba': 'yo', 'Zulu': 'zu'}
activate_google_translate = False #@param {type:"boolean"}
your_language = "French" # @param ['Afrikaans', 'Albanian', 'Amharic', 'Arabic', 'Armenian', 'Azerbaijani', 'Basque', 'Belarusian', 'Bengali', 'Bosnian', 'Bulgarian', 'Catalan', 'Cebuano', 'Chinese (Simplified)', 'Chinese (Traditional)', 'Corsican', 'Croatian', 'Czech', 'Danish', 'Dutch', 'English', 'Esperanto', 'Estonian', 'Finnish', 'French', 'Frisian', 'Galician', 'Georgian', 'German', 'Greek', 'Gujarati', 'Haitian Creole', 'Hausa', 'Hawaiian', 'Hebrew', 'Hindi', 'Hmong', 'Hungarian', 'Icelandic', 'Igbo', 'Indonesian', 'Irish', 'Italian', 'Japanese', 'Javanese', 'Kannada', 'Kazakh', 'Khmer', 'Korean', 'Kurdish', 'Kyrgyz', 'Lao', 'Latin', 'Latvian', 'Lithuanian', 'Luxembourgish', 'Macedonian', 'Malagasy', 'Malay', 'Malayalam', 'Maltese', 'Maori', 'Marathi', 'Mongolian', 'Myanmar (Burmese)', 'Nepali', 'Norwegian', 'Nyanja (Chichewa)', 'Pashto', 'Persian', 'Polish', 'Portuguese (Portugal, Brazil)', 'Punjabi', 'Romanian', 'Russian', 'Samoan', 'Scots Gaelic', 'Serbian', 'Sesotho', 'Shona', 'Sindhi', 'Sinhala (Sinhalese)', 'Slovak', 'Slovenian', 'Somali', 'Spanish', 'Sundanese', 'Swahili', 'Swedish', 'Tagalog (Filipino)', 'Tajik', 'Tamil', 'Telugu', 'Thai', 'Turkish', 'Ukrainian', 'Urdu', 'Uzbek', 'Vietnamese', 'Welsh', 'Xhosa', 'Yiddish', 'Yoruba', 'Zulu']

# Applying the selected language
j = json.loads(open('settings-colab-template.json', 'r').read())
j["google_translate-language string"] = language_codes[your_language]
with open('settings-colab.json', 'w') as f:
  f.write(json.dumps(j, indent=4))

if activate_google_translate:
  !source /usr/local/bin/activate && conda activate textgen && python server.py --share --cai-chat --auto-devices --model $model_name --settings settings-colab.json --extensions google_translate --no-stream
else:
  !source /usr/local/bin/activate && conda activate textgen && python server.py --share --cai-chat --auto-devices --model $model_name --settings settings-colab.json
